In [1]:
from mlx_lm import load, generate

# model, tokenizer = load("jerryzhao173985/Qwen2.5-7B-poem-8bit")

# messages = [{"role": "user", "content": prompt}]
# prompt = tokenizer.apply_chat_template(
#     messages, add_generation_prompt=True
# )
# response = generate(model, tokenizer, prompt=prompt, verbose=True)
# response

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

In [2]:
import json

with open("poems_features.json", "r", encoding="utf-8") as f:
    poems = json.load(f)

with open("augmented_poems_o1_mini.json", "r", encoding="utf-8") as f:
    generated_poems = json.load(f)

# Ensure both lists are of the same length
if len(poems) != len(generated_poems):
    raise ValueError("The two lists must have the same number of items.")

for poem, generated_poem in zip(poems, generated_poems):
    print(poem.get('title'), generated_poem.get('original_title'), generated_poem.get('title'))
    assert(generated_poem.get('original_title')==None or poem.get('title') == generated_poem.get('original_title'))
    # Process each corresponding pair
    # Example: print titles if they have a 'title' key
    # print(f"Original: {poem.get('title')}, Generated: {generated_poem.get('title')}")

河 河 流
雨的碎片 雨的碎片 风的低语
湖 湖 寒塘
等待与沉溺 等待与沉溺 迷失与静寂
桥头的抗争 桥头的抗争 桥边的融合
路边花开 路边花开 车辙花语
冬至 冬至 融雪
冬日的花序 冬日的花序 冬日新生
风吹云散，寻觅之影 风吹云散，寻觅之影 风带走云影
死亡 死亡 终章
呼吸之间 呼吸之间 呼吸边缘
忍冬的重量 忍冬的重量 月影的重量
利刃 利刃 锋影
雨的穿透 雨的穿透 雨幕中的重生
猫山 猫山 静山
厌恶 厌恶 自我拷问
太阳与蜜糖 太阳与蜜糖 光影之间
冻死的蜜蜂 None Untitled Expansion
痛苦的气泡 痛苦的气泡 痛苦的微光
光的蒸发 光的蒸发 月光的幻化
呼吸的形状 呼吸的形状 气息的轮廓
隔阂与屏障 隔阂与屏障 隔阂与屏障
癌症的夜晚 癌症的夜晚 康复的晨曦
血与树 血与树 剑与花
星星 星星 消逝
夜晚的抚慰 None Untitled Expansion
小石块 小石块 星尘
天空的裂缝 天空的裂缝 裂隙中的孤光
海与森林的沉默 海与森林的沉默 大海与树林的静默
火焰 火焰 燃烧的脉络
今夜的我 今夜的我 孤夜之下
黑夜 黑夜 夜幕下
雅鲁藏布江 雅鲁藏布江 长河守望
林芝卖水果的女人 林芝卖水果的女人 林芝卖水果的女人
逃离 逃离 离开地球的渴望
自由的雨 自由的雨 奔腾的风
安宁 安宁 沉静
被虫蛀的思想 被虫蛀的思想 心灵的蚀刻
扼住那些野花的喉咙 扼住那些野花的喉咙 掩盖黎明的阴影
记忆的味道 记忆的味道 回忆的余韵
平静的湖水 平静的湖水 永恒的森林
父亲的笨拙之爱 父亲的笨拙之爱 父亲沉默的力量
行囊 行囊 沉重的行囊
救赎之路 救赎之路 穿越黎明
回家的意义 回家的意义 回家的意义
漂泊与选择 漂泊与选择 漂泊与选择
远方的故乡 远方的故乡 远方的故乡
命运的定义 命运的定义 身份的枷锁
觉醒与平等 觉醒与平等 自由的回响
爱与本真 爱与本真 本真的爱
一生的瞬间 一生的瞬间 生命中的光芒
幸福的祈愿 幸福的祈愿 幸福的祈愿
传说的风景 传说的风景 追寻的航程
束缚的生命 束缚的生命 沉寂的束缚
书店的命运 书店的命运 书架的邂逅
污泥 污泥 深泥
大山 大山 巍峨之路
沙场 沙场 沙场
功名与归隐 功名与归隐 岁暮归心
魔方 魔方 遗失的魔方
妹妹的世界 妹妹的世界 色彩的真相
在法兰德斯的罂粟花 在法兰德

In [3]:
def get_imagery(item):
    """
    Retrieve the 'imagery' list from a dictionary.
    If 'imagery' doesn't exist or is None, return an empty list.
    """
    imagery = item.get('imagery')
    if isinstance(imagery, list):
        return imagery
    else:
        return []

# Ensure both lists are iterated up to the length of the longer list
from itertools import zip_longest

union_imagery = []
for p, gp in zip_longest(poems, generated_poems, fillvalue={}):
    p_imagery = get_imagery(p)
    gp_imagery = get_imagery(gp)
    # Perform union using sets
    combined_set = set(p_imagery).union(gp_imagery)
    union_imagery.append(list(combined_set))

# # Display the result
# for idx, imagery in enumerate(union_imagery):
#     print(f"Union for item {idx}: {imagery}")


In [4]:
PROMPT = """<|im_start|>system 用户会给出一个主题，请按照给定的主题，切实准确简洁且情感丰富地写一首现代诗<|im_end|> <|im_start|>user {image}<|im_end|> <|im_start|>assistant"""

In [6]:
import json
from pathlib import Path

def find_adapters_and_configs(start_dir='./'):
    """
    Searches for 'adapters.safetensors' files starting from 'start_dir' and
    collects their directories. Also loads the corresponding 'adapter_config.json'
    files into a dictionary.

    Args:
        start_dir (str or Path): The directory to start the search from.

    Returns:
        tuple:
            - adaptors (list): List of directories containing 'adapters.safetensors'.
            - adapter_configs (dict): Mapping from directory paths to their config data.
    """
    start_path = Path(start_dir).resolve()
    adaptors = []
    adapter_configs = {}

    # Use rglob to recursively search for 'adapters.safetensors'
    for adapter_file in start_path.rglob('adapters.safetensors'):
        dir_path = adapter_file.parent
        adaptors.append(str(dir_path.relative_to(start_path)))

        # Path to 'adapter_config.json' in the same directory
        config_file = dir_path / 'adapter_config.json'

        if config_file.exists():
            try:
                with config_file.open('r', encoding='utf-8') as f:
                    config_data = json.load(f)
                adapter_configs[str(dir_path.relative_to(start_path))] = config_data
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in {config_file}: {e}")
            except Exception as e:
                print(f"Unexpected error reading {config_file}: {e}")
        else:
            print(f"Warning: 'adapter_config.json' not found in {dir_path}")

    return adaptors, adapter_configs


adaptors, adapter_configs = find_adapters_and_configs('./')

print("List of directories containing 'adapters.safetensors':")
for adaptor in adaptors:
    print(f"- {adaptor}")

print("\nAdapter Configurations:")
for path, config in adapter_configs.items():
    print(f"\nDirectory: {path}")
    print(json.dumps(config, indent=4))


In [7]:
import json
from pathlib import Path

def find_adapters_and_configs(start_dir='./', exclude_dirs=None, keys_to_keep=None):
    """
    Searches for 'adapters.safetensors' files starting from 'start_dir' and
    collects their directories, excluding any specified in 'exclude_dirs'.
    Also loads the corresponding 'adapter_config.json' files into a dictionary
    with only the specified keys retained.

    Args:
        start_dir (str or Path): The directory to start the search from.
        exclude_dirs (list, optional): List of directory names to exclude from the results.
        keys_to_keep (list, optional): List of keys to retain in the configuration.

    Returns:
        tuple:
            - adaptors (list): List of directories containing 'adapters.safetensors', excluding specified dirs.
            - adapter_configs (dict): Mapping from directory paths to their filtered config data.
    """
    if exclude_dirs is None:
        exclude_dirs = []
    if keys_to_keep is None:
        keys_to_keep = [
            "grad_checkpoint",
            "iters",
            "learning_rate",
            "lora_parameters",
            "lr_schedule",
            "model",
            "num_layers"
        ]

    start_path = Path(start_dir).resolve()
    adaptors = []
    adapter_configs = {}

    # Use rglob to recursively search for 'adapters.safetensors'
    for adapter_file in start_path.rglob('adapters.safetensors'):
        dir_path = adapter_file.parent
        relative_dir = str(dir_path.relative_to(start_path))

        # Check if the directory is in the exclude list
        if relative_dir in exclude_dirs:
            print(f"Excluding directory: {relative_dir}")
            continue

        adaptors.append(relative_dir)

        # Path to 'adapter_config.json' in the same directory
        config_file = dir_path / 'adapter_config.json'

        if config_file.exists():
            try:
                with config_file.open('r', encoding='utf-8') as f:
                    config_data = json.load(f)
                # Filter the config to keep only the specified keys
                filtered_config = {key: config_data[key] for key in keys_to_keep if key in config_data}
                adapter_configs[relative_dir] = filtered_config
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in {config_file}: {e}")
            except Exception as e:
                print(f"Unexpected error reading {config_file}: {e}")
        else:
            print(f"Warning: 'adapter_config.json' not found in {dir_path}")

    return adaptors, adapter_configs


# Specify directories to exclude
directories_to_exclude = [
    "adapters-xiaoxue-poem-cn",
    "adapters copy"
]

# Specify keys to retain in the configuration
keys_of_interest = [
    "grad_checkpoint",
    "iters",
    "learning_rate",
    "lora_parameters",
    "lr_schedule",
    "model",
    "num_layers"
]

adaptors, adapter_configs = find_adapters_and_configs(
    start_dir='./',
    exclude_dirs=directories_to_exclude,
    keys_to_keep=keys_of_interest
)

print("\nList of directories containing 'adapters.safetensors':")
for adaptor in adaptors:
    print(f"- {adaptor}")

print("\nAdapter Configurations:")
for path, config in adapter_configs.items():
    print(f"\nDirectory: {path}")
    print(json.dumps(config, indent=4))


Excluding directory: adapters copy
Excluding directory: adapters-xiaoxue-poem-cn

List of directories containing 'adapters.safetensors':
- adapters_lora_1
- adapters
- adapters copy 2
- adapters-before
- adapters-Jan21
- adapters-bf16-2400epochs
- adapters_lora_4
- adapters_lora_3
- adapters1
- adapters_lora_2

Adapter Configurations:

Directory: adapters_lora_1
{
    "grad_checkpoint": false,
    "iters": 800,
    "learning_rate": 1e-05,
    "lora_parameters": {
        "keys": [
            "self_attn.q_proj",
            "self_attn.v_proj",
            "mlp.down_proj",
            "mlp.gate_proj",
            "mlp.up_proj"
        ],
        "rank": 16,
        "alpha": 16,
        "dropout": 0.05,
        "scale": 8.0
    },
    "lr_schedule": {
        "name": "cosine_decay",
        "warmup": 100,
        "warmup_init": 1e-07,
        "arguments": [
            1e-05,
            800,
            1e-07
        ]
    },
    "model": "mlx-community/Qwen2.5-7B-Instruct-bf16",
    "n

In [10]:
import json
from pathlib import Path

def find_adapters_and_configs(start_dir='./', exclude_dirs=None, keys_to_keep=None):
    """
    Searches for 'adapters.safetensors' files starting from 'start_dir' and
    collects their directories, excluding any specified in 'exclude_dirs'.
    Also loads the corresponding 'adapter_config.json' files into a dictionary
    with only the specified keys retained.

    Args:
        start_dir (str or Path): The directory to start the search from.
        exclude_dirs (list, optional): List of directory names to exclude from the results.
        keys_to_keep (list, optional): List of keys to retain in the configuration.

    Returns:
        tuple:
            - adaptors (list): List of directories containing 'adapters.safetensors', excluding specified dirs.
            - adapter_configs (dict): Mapping from directory paths to their filtered config data.
    """
    if exclude_dirs is None:
        exclude_dirs = []
    if keys_to_keep is None:
        keys_to_keep = [
            "grad_checkpoint",
            "iters",
            "learning_rate",
            "lora_parameters",
            "lr_schedule",
            "model",
            "num_layers"
        ]

    start_path = Path(start_dir).resolve()
    adaptors = []
    adapter_configs = {}

    # Recursively search for 'adapters.safetensors'
    for adapter_file in start_path.rglob('adapters.safetensors'):
        dir_path = adapter_file.parent
        relative_dir = str(dir_path.relative_to(start_path))

        # Exclude specified directories
        if relative_dir in exclude_dirs:
            print(f"Excluding directory: {relative_dir}")
            continue

        # Path to 'adapter_config.json' in the same directory
        config_file = dir_path / 'adapter_config.json'

        if config_file.exists():
            try:
                with config_file.open('r', encoding='utf-8') as f:
                    config_data = json.load(f)
                # Retain only specified keys
                filtered_config = {key: config_data[key] for key in keys_to_keep if key in config_data}
                adapter_configs[relative_dir] = filtered_config
                adaptors.append(relative_dir)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in {config_file}: {e}")
            except Exception as e:
                print(f"Unexpected error reading {config_file}: {e}")
        else:
            print(f"Warning: 'adapter_config.json' not found in {dir_path}")

    return adaptors, adapter_configs

def filter_configs(adapter_configs, conditions):
    """
    Filters the adapter configurations based on specified conditions.

    Args:
        adapter_configs (dict): Mapping from directory paths to their config data.
        conditions (dict): Dictionary where keys are config keys and values are functions that take the config value and return True/False.

    Returns:
        dict: Filtered adapter_configs that satisfy all conditions.
    """
    filtered_configs = {}
    for dir_path, config in adapter_configs.items():
        meets_conditions = True
        for key, condition in conditions.items():
            if key not in config:
                # If a key is missing, consider the condition as not met
                meets_conditions = False
                break
            if not condition(config[key]):
                # If the condition is not met, exclude this config
                meets_conditions = False
                break
        if meets_conditions:
            filtered_configs[dir_path] = config
    return filtered_configs

#  # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# Directories to exclude from the search
directories_to_exclude = [
    "adapters-xiaoxue-poem-cn",
    "adapters copy"
]

# Keys to retain from each adapter's configuration
keys_of_interest = [
    "grad_checkpoint",
    "iters",
    "learning_rate",
    "lora_parameters",
    "lr_schedule",
    "model",
    "num_layers"
]

# Define the filtering conditions
# Each key corresponds to a configuration key, and the lambda defines the condition
filter_conditions = {
    "model": lambda x: "bf16" in x.lower(),
    "grad_checkpoint": lambda x: x is False,
    "learning_rate": lambda x: x <= 1e-06,
    "iters": lambda x: x >= 600
}

# Step 1: Find all adapters and their configurations
adaptors, adapter_configs = find_adapters_and_configs(
    start_dir='./',
    exclude_dirs=directories_to_exclude,
    keys_to_keep=keys_of_interest
)

# Step 2: Apply filtering conditions to the configurations
filtered_adapter_configs = filter_configs(adapter_configs, filter_conditions)

# Step 3: Update the adaptors list to include only those that passed the filter
filtered_adaptors = list(filtered_adapter_configs.keys())

# Display the filtered results
print("\nList of directories containing 'adapters.safetensors' after filtering:")
for adaptor in filtered_adaptors:
    print(f"- {adaptor}")

print("\nFiltered Adapter Configurations:")
for path, config in filtered_adapter_configs.items():
    print(f"\nDirectory: {path}")
    print(json.dumps(config, indent=4))


Excluding directory: adapters copy
Excluding directory: adapters-xiaoxue-poem-cn

List of directories containing 'adapters.safetensors' after filtering:
- adapters-Jan21
- adapters-bf16-2400epochs
- adapters_lora_4
- adapters_lora_3

Filtered Adapter Configurations:

Directory: adapters-Jan21
{
    "grad_checkpoint": false,
    "iters": 1000,
    "learning_rate": 1e-06,
    "lora_parameters": {
        "rank": 8,
        "alpha": 16,
        "dropout": 0.0,
        "scale": 10.0
    },
    "lr_schedule": null,
    "model": "mlx-community/Qwen2.5-7B-Instruct-bf16",
    "num_layers": 16
}

Directory: adapters-bf16-2400epochs
{
    "grad_checkpoint": false,
    "iters": 1200,
    "learning_rate": 1e-06,
    "lora_parameters": {
        "rank": 8,
        "alpha": 16,
        "dropout": 0.0,
        "scale": 10.0
    },
    "lr_schedule": null,
    "model": "mlx-community/Qwen2.5-7B-Instruct-bf16",
    "num_layers": 16
}

Directory: adapters_lora_4
{
    "grad_checkpoint": false,
    "iter

In [11]:
adaptor_results = {}

for adaptor in adaptors:
    results = []
    model, tokenizer = load("jerryzhao173985/Qwen2.5-7B-poem-8bit", adapter_path=adaptor)
    for images in union_imagery:
        res = []
        for image in images:
            messages = [{"role": "user", "content": PROMPT.format(image = image)}]
            prompt = tokenizer.apply_chat_template(
                messages, add_generation_prompt=True
            )
            response = generate(model, tokenizer, prompt=prompt, verbose=True)
            res.append([image, response])
        results.append(res)
    adaptor_results[adaptor] = results

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

在夜的深处
虫子用尽全部的力气
嘶喊着什么
Prompt: 65 tokens, 107.264 tokens-per-sec
Generation: 18 tokens, 30.382 tokens-per-sec
Peak memory: 16.190 GB
我试图通过一片叶子
得到冬天的温度
Prompt: 64 tokens, 420.102 tokens-per-sec
Generation: 11 tokens, 40.158 tokens-per-sec
Peak memory: 16.190 GB
在流动中冻结utowater
Prompt: 65 tokens, 521.636 tokens-per-sec
Generation: 7 tokens, 42.512 tokens-per-sec
Peak memory: 16.190 GB
山峰
是被压碎了的
玻璃
Prompt: 64 tokens, 683.397 tokens-per-sec
Generation: 12 tokens, 39.825 tokens-per-sec
Peak memory: 16.190 GB
让我一起来融化一片雪花
Prompt: 64 tokens, 689.433 tokens-per-sec
Generation: 6 tokens, 43.843 tokens-per-sec
Peak memory: 16.190 GB
那些血迹风干在冷风里
Prompt: 64 tokens, 690.464 tokens-per-sec
Generation: 10 tokens, 40.515 tokens-per-sec
Peak memory: 16.190 GB
轻舞的rze
Prompt: 68 tokens, 540.691 tokens-per-sec
Generation: 5 tokens, 44.978 tokens-per-sec
Peak memory: 16.190 GB
风中的落叶，不是没有根的人
Prompt: 66 tokens, 519.447 tokens-per-sec
Generation: 9 tokens, 40.257 tokens-per-sec
Peak memory: 16.190 GB
在碎裂的玻璃上
长出的全

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

一片被遗忘的角落
这些没有脚只轮子的虫子
带着我们驶出
这是完全属于他们的天地
Prompt: 65 tokens, 368.264 tokens-per-sec
Generation: 28 tokens, 31.872 tokens-per-sec
Peak memory: 16.332 GB
在落尽了叶子的叶子树下
在已经消解了温度的空气中
Prompt: 64 tokens, 530.168 tokens-per-sec
Generation: 19 tokens, 36.525 tokens-per-sec
Peak memory: 16.332 GB
在夜晚
集一片岩石喝掉他
Prompt: 65 tokens, 519.441 tokens-per-sec
Generation: 10 tokens, 38.454 tokens-per-sec
Peak memory: 16.332 GB
我是峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰峰
Prompt: 64 tokens, 680.536 tokens-per-sec
Generation: 256 tokens, 34.469 tokens-per-sec
Peak memory: 16.332 GB
让我给你看一幅画
Prompt: 64 tokens, 679.463 tokens-per-sec
Generation: 6 tokens, 39.781 tokens-per-sec
Peak memory: 16.332 GB
让那些
在一起
冻死的死掉
Prompt: 64 tokens, 671.233 tokens-per-sec
Generation: 12 tokens, 37.624 tokens-per-sec
Peak

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

深黑的夜中
发出几束光
Prompt: 65 tokens, 493.851 tokens-per-sec
Generation: 12 tokens, 38.354 tokens-per-sec
Peak memory: 16.332 GB
在浮雪的里面
Prompt: 64 tokens, 766.935 tokens-per-sec
Generation: 6 tokens, 61.221 tokens-per-sec
Peak memory: 16.332 GB
在需要的时间段里消逝
在必要的时候消失
Prompt: 65 tokens, 560.054 tokens-per-sec
Generation: 13 tokens, 55.283 tokens-per-sec
Peak memory: 16.332 GB
山峰
会
消失
在
峰峰峰峰峰
峰峰峰峰峰一起
峰峰峰一起
峰一起
起一起
一起一起一起一起一起一起一起一起一起一起一起一起
Prompt: 64 tokens, 766.108 tokens-per-sec
Generation: 46 tokens, 52.211 tokens-per-sec
Peak memory: 16.332 GB
将自己以及未来的所有，
扑在在一起
Prompt: 64 tokens, 768.006 tokens-per-sec
Generation: 10 tokens, 56.383 tokens-per-sec
Peak memory: 16.332 GB
冷风为我的生命画上退出IGHLIGHTS
Prompt: 64 tokens, 767.558 tokens-per-sec
Generation: 11 tokens, 56.393 tokens-per-sec
Peak memory: 16.332 GB
那些不复存在的东西semblerait-ils
peiropoienièrés
awiaryés
lévyanés
fëurénièrés
Prompt: 68 tokens, 584.762 tokens-per-sec
Generation: 37 tokens, 52.508 tokens-per-sec
Peak memory: 16.332 GB
风中的风干的风
风中的风
风中的叶
飞远

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

IndexError: list index out of range

In [23]:
adaptor_results.keys()
# for key in adaptor_results.keys():
#     print(len(adaptor_results[key]))
adapter_configs.keys()

dict_keys(['adapters_lora_1', 'adapters', 'adapters copy 2', 'adapters-before', 'adapters-Jan21', 'adapters-bf16-2400epochs', 'adapters_lora_4', 'adapters_lora_3', 'adapters1', 'adapters_lora_2'])

In [25]:
# Current implementation NEW!


In [27]:
import json
import os
import pandas as pd
from pathlib import Path

# Assuming the necessary functions 'load' and 'generate' are defined elsewhere
from mlx_lm import load, generate

# Flatten the adaptor_results and include adaptor name
flattened_results = []

for adaptor, results in adaptor_results.items():
    for outer_idx, inner_list in enumerate(results):
        for inner_idx, pair in enumerate(inner_list):
            flattened_results.append({
                'adaptor': adaptor,
                'outer_idx': outer_idx,
                'inner_idx': inner_idx,
                'image': pair[0],
                'response': pair[1],
                'accepted': False,  # To track acceptance
                'edited': False     # To track edits
            })

# # Optional: Combine with adapter_configs
# # If you want to include adapter configurations in each entry
# for entry in flattened_results:
#     adaptor = entry['adaptor']
#     entry['adapter_config'] = adapter_configs.get(adaptor, {})

# Save the flattened results to a JSON file
output_json_path = "overall_poems.json"

with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(flattened_results, f, indent=2, ensure_ascii=False)

print(f"All results have been saved to '{output_json_path}'.")

# Optional: If you want to append new results to an existing JSON
def append_to_json(new_data, json_file):
    """
    Appends new_data (a list of dictionaries) to an existing JSON file.

    Args:
        new_data (list): List of dictionaries to append.
        json_file (str): Path to the JSON file.
    """
    if os.path.exists(json_file):
        with open(json_file, 'r', encoding='utf-8') as f:
            existing_data = json.load(f)
        if isinstance(existing_data, list):
            existing_data.extend(new_data)
        else:
            print(f"Unexpected data format in {json_file}. Expected a list.")
            return
    else:
        existing_data = new_data

    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(existing_data, f, indent=2, ensure_ascii=False)
    print(f"Data has been appended to '{json_file}'.")

# Example of appending to an existing JSON file
# append_to_json(flattened_results, "overall_poems.json")


All results have been saved to 'overall_poems.json'.


In [ ]:

# The following code processes the saved JSON file to clean and structure the data

# Define the path to the JSON file
json_file_path = 'overall_poems.json'

# Optional: Define the path for the cleaned JSON output
cleaned_json_file_path = 'cleaned_poems.json'

# Check if the file exists
if not os.path.exists(json_file_path):
    raise FileNotFoundError(f"The file {json_file_path} does not exist in the current directory.")

# Load the JSON data
with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Display the type of data loaded
print(f"Data type loaded: {type(data)}")

# Display the first few elements to understand the structure
print("Sample data:")
for i, entry in enumerate(data[:2], start=1):
    print(f"\nEntry {i}:")
    print(entry)

# Initialize an empty list to hold the cleaned data
cleaned_data = []

for entry in data:
    # Ensure that each entry has the required keys
    required_keys = ['adaptor', 'image', 'response']
    if all(key in entry for key in required_keys):
        title = entry['adaptor']  # Using adaptor as the title
        body = entry['response']  # Using response as the body
        # Check that neither title nor body is empty
        if title.strip() and body.strip():
            cleaned_data.append({'title': title.strip(), 'body': body.strip()})
    else:
        print(f"Skipping invalid entry: {entry}")

# Display the number of valid entries
print(f"\nNumber of valid [title, body] pairs: {len(cleaned_data)}")

# Remove any empty dictionaries if present
cleaned_data = [entry for entry in cleaned_data if entry]

print(f"Number of entries after removing empty lists: {len(cleaned_data)}")

# Create a DataFrame from the cleaned data
df = pd.DataFrame(cleaned_data)

# Display the first few rows of the DataFrame
print("\nFirst few rows of the DataFrame:")
print(df.head())

# Save the cleaned DataFrame to a new JSON file
df.to_json(cleaned_json_file_path, orient='records', force_ascii=False, indent=4)

print(f"Cleaned data has been saved to '{cleaned_json_file_path}'.")

# Load and display the cleaned JSON data
with open(cleaned_json_file_path, 'r', encoding='utf-8') as file:
    cleaned_json_data = json.load(file)

# Display the first few entries
print("\nSample cleaned data:")
for entry in cleaned_json_data[:3]:
    print(entry)

In [12]:
results[0], results[1], results[2][0], results[2][1]

IndexError: list index out of range

In [32]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
import ipywidgets as widgets
from IPython.display import display, clear_output
from itertools import chain
import copy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
# Flatten the 'results' list and keep track of original positions
flattened_results = []
for outer_idx, inner_list in enumerate(results):
    for inner_idx, pair in enumerate(inner_list):
        flattened_results.append({
            'outer_idx': outer_idx,
            'inner_idx': inner_idx,
            'image': pair[0],
            'response': pair[1],
            'accepted': False,  # To track acceptance
            'edited': False     # To track edits
        })
        
import json

with open("accept_poems.json", "w", encoding="utf-8") as f:
    json.dump(flattened_results, f, indent=2, ensure_ascii=False)


In [36]:
# After python tk appp is done and reviewed the [poems and save 
# view it here and save concise version
import json
import pandas as pd
import os


In [37]:
# Define the path to the JSON file
json_file_path = 'updated_poems.json'

# Optional: Define the path for the cleaned JSON output
cleaned_json_file_path = 'cleaned_poems.json'

# Check if the file exists
if not os.path.exists(json_file_path):
    raise FileNotFoundError(f"The file {json_file_path} does not exist in the current directory.")

# Load the JSON data
with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Display the type of data loaded
print(f"Data type loaded: {type(data)}")

# Display the first few elements to understand the structure
print("Sample data:")
for i, outer_list in enumerate(data[:2], start=1):
    print(f"\nList {i}:")
    for pair in outer_list[:2]:
        print(pair)


Data type loaded: <class 'list'>
Sample data:

List 1:
['', '']
['河水', '在夜的深处\n河水\n是黑暗的\n眼睛']

List 2:
['', '']
['雨水中的伞', '在雨的细语中\n我撑开伞\n遮挡了\n从天而降的思念']


In [39]:
# Initialize an empty list to hold the flattened data
flattened_data = []

for outer_list in data:
    for pair in outer_list:
        # Ensure that each pair has exactly two elements: title and body
        if isinstance(pair, list) and len(pair) == 2:
            title, body = pair
            # Check that neither title nor body is empty
            if title.strip() and body.strip():
                flattened_data.append({'title': title.strip(), 'body': body.strip()})
        else:
            print(f"Skipping invalid pair: {pair}")

# Display the number of valid entries
print(f"\nNumber of valid [title, body] pairs: {len(flattened_data)}")

# Remove any empty dictionaries if present
flattened_data = [entry for entry in flattened_data if entry]

print(f"Number of entries after removing empty lists: {len(flattened_data)}")



Number of valid [title, body] pairs: 67
Number of entries after removing empty lists: 67


In [40]:
# Create a DataFrame from the flattened data
df = pd.DataFrame(flattened_data)

# Display the first few rows of the DataFrame
df.head()


,title,body
0,河水,在夜的深处\n河水\n是黑暗的\n眼睛
1,积雪,在落雪的夜晚\n我将积雪\n藏在了心里
2,虫子,在夜的深处\n虫子\n用尽全身的力气\n嘶喊\n直到\n沙哑
3,山峰,在山峰的顶端\n我将自己\n压成一片叶子
4,雨水中的伞,在雨的细语中\n我撑开伞\n遮挡了\n从天而降的思念


In [41]:
# Save the cleaned DataFrame to a new JSON file
df.to_json(cleaned_json_file_path, orient='records', force_ascii=False, indent=4)

print(f"Cleaned data has been saved to '{cleaned_json_file_path}'.")


Cleaned data has been saved to 'cleaned_poems.json'.


In [42]:
# Load and display the cleaned JSON data
with open(cleaned_json_file_path, 'r', encoding='utf-8') as file:
    cleaned_data = json.load(file)

# Display the first few entries
print("\nSample cleaned data:")
for entry in cleaned_data[:3]:
    print(entry)



Sample cleaned data:
{'title': '河水', 'body': '在夜的深处\n河水\n是黑暗的\n眼睛'}
{'title': '积雪', 'body': '在落雪的夜晚\n我将积雪\n藏在了心里'}
{'title': '虫子', 'body': '在夜的深处\n虫子\n用尽全身的力气\n嘶喊\n直到\n沙哑'}


In [43]:
from IPython.display import display, HTML

# Convert DataFrame to HTML and display
display(HTML(df.to_html(index=False, escape=False)))


title,body
河水,在夜的深处\n河水\n是黑暗的\n眼睛
积雪,在落雪的夜晚\n我将积雪\n藏在了心里
虫子,在夜的深处\n虫子\n用尽全身的力气\n嘶喊\n直到\n沙哑
山峰,在山峰的顶端\n我将自己\n压成一片叶子
雨水中的伞,在雨的细语中\n我撑开伞\n遮挡了\n从天而降的思念
枯萎的莲花,在枯萎的莲花里\n藏着一个\n干涸的\n海
凝视的寒塘,凝视的寒塘\n是落尽繁华的枝头
冬日的寒冷,冬日的寒冷\n是冰封的泪滴\n在夜的黑幕下\n静静的流淌
枯榕的影子,在褪色的黄昏\n我与枯榕的影子\n在废弃的街道上\n一起流浪
冰冷的水,冰凉的水\n在夜晚静静的流\n它知道\n它流过的地方\n什么都不会知道


In [27]:
# messages = [{"role": "user", "content": prompt}]
# prompt = tokenizer.apply_chat_template(
#     messages, add_generation_prompt=True
# )
# response = generate(model, tokenizer, prompt=prompt, verbose=True)